<a href="https://colab.research.google.com/github/hail-members/llm-based-services/blob/main/Chapter_5_%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Train 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json

# Dev 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

import json

# Train 데이터 로드
with open("KorQuAD_v1.0_train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Dev 데이터 로드
with open("KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# 데이터 구조 확인
print("Train Data Keys:", train_data.keys())
print("Example Data:", train_data["data"][0])  # 첫 번째 문단 출력

--2025-04-04 01:13:39--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  11.4MB/s    in 3.2s    

2025-04-04 01:13:42 (11.4 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-04-04 01:13:42--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[=====

In [7]:
# 라이브러리 임포트
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader


# KoGPT2 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
# 데이터셋 정의 (질문과 답변을 GPT2 입력 형식으로 변환)
class KorQuADDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data["data"]
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.total_qas = []  # 모든 질문-답변 쌍을 저장

        # 전체 질문-답변 쌍을 리스트로 저장
        for article in self.data:
            for paragraph in article["paragraphs"]:
                for qa in paragraph["qas"]:
                    self.total_qas.append((paragraph["context"], qa))

    def __len__(self):
        return len(self.total_qas)

    def __getitem__(self, idx):
        # 질문과 답변 추출
        context, qa = self.total_qas[idx]
        question = qa["question"]
        answer_text = qa["answers"][0]["text"]

        # GPT2 입력 텍스트 생성 (질문 + 컨텍스트 + 답변)
        # input_text = f"<usr> {question} <sys> {context} </s>"
        # GPT2 입력 텍스트 생성 (질문+ 답변)
        input_text = f"<usr> {question} </s>"
        target_text = f"<sys> {answer_text} </s>"

        # 토큰화 및 인코딩
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding="max_length")
        target_ids = self.tokenizer.encode(target_text, max_length=self.max_length, truncation=True, padding="max_length")

        return {
            "input_ids": torch.tensor(input_ids),
            "labels": torch.tensor(target_ids)
        }

train_dataset = KorQuADDataset(train_data, tokenizer)
dev_dataset = KorQuADDataset(dev_data, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=4)


In [26]:
# 저장된 데이터 보기
for batch in train_dataloader:
    print(batch)
    print(tokenizer.decode(batch["input_ids"][0], skip_special_tokens=False))
    print(tokenizer.decode(batch["labels"][0], skip_special_tokens=False))
    break

{'input_ids': tensor([[    2, 14603, 20293,  ...,     3,     3,     3],
        [    2,  9089,  8170,  ...,     3,     3,     3],
        [    2,  9724, 39514,  ...,     3,     3,     3],
        [    2, 17867, 29191,  ...,     3,     3,     3]]), 'labels': tensor([[    4,  9018,  8785,  ...,     3,     3,     3],
        [    4,  9664,  7791,  ...,     3,     3,     3],
        [    4, 17797,   739,  ...,     3,     3,     3],
        [    4,  9077,  7374,  ...,     3,     3,     3]])}
<usr> 1994년 강경식 교수의 기고문에 따르면 이휘소 일가족의 사고당시 그 경위를 강경식 교수에게 알려주었던 인물은 누구인가? <sys> 《과학과 기술》 1994년 1월호에 실린 〈내가 아는 고 이휘소 박사〉라는 강경식 전 브라운 대학교 교수의 특별기고문에는 당시 이휘소의 비서가 사고 직후 강경식에게 전화를 걸어 설명한 사고 당시 상황이 실려 있다. 이휘소는 1977년 6월 16일 12시가 되기 전에 가족들을 태우고 콜로라도 주 아스펜 시로 출발했고, 그로부터 약 1시간 30분 간 일리노이 주 내의 고속도로 I-80의 아이오와 주 경계로부터 약 30마일 떨어진 지점까지 정규속도 55마일로 운전해 가고 있었다. 그러다 오후 1시 22분 경, 건너편 내부고속도로선을 동쪽으로 달리던 대형 트레일러의 타이어가 터지면서 중심을 잃어 조정을 못하고 중앙분리지역을 넘어와 서쪽으로 달리고 있던 이휘소의 차량의 운전석을 덮쳤다. 이 사고로 이휘소 가족들은 가볍게 다쳤지만, 본인은 즉사했다. 한편, 사고 경위

In [11]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 문장 생성 함수 정의
def generate_sentence(model, seed_text, max_length=50):
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)
    gen_ids = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True,
    )
    generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    return generated_text

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")


질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란?   
, , . (중략) 이번에 출시된 신제품은 '스마트폰용 스마트패드'다.
이 제품은 스마트폰을 통해 다양한 콘텐츠를 즐길 수 있는 것이 특징이다.
특히 기존 제품보다 최대 2배 이상 빠른 속도로 데이터를 전송


In [15]:

from torch.optim import AdamW
from tqdm import tqdm

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-3)

# 학습 루프 정의 (tqdm으로 Progress Bar 추가)
epochs = 5
model.train()

# 조기 종료 조건 설정
# max_batches_per_epoch = 1000  # 한 에포크에서 최대 실행할 배치 수

for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    print(f"Epoch {epoch + 1} completed. Average Loss: {epoch_loss / (batch_idx + 1)}")

# 모델 저장
model.save_pretrained("./kogpt2-korquad-finetuned")
tokenizer.save_pretrained("./kogpt2-korquad-finetuned")

Epoch 1: 100%|██████████| 15102/15102 [36:14<00:00,  6.95it/s, Batch Loss=0.0548] 


Epoch 1 completed. Average Loss: 0.06806645459783972


Epoch 2: 100%|██████████| 15102/15102 [36:20<00:00,  6.92it/s, Batch Loss=0.0728] 


Epoch 2 completed. Average Loss: 0.07070691338127917


Epoch 3: 100%|██████████| 15102/15102 [36:20<00:00,  6.93it/s, Batch Loss=0.0685] 


...

Epoch 4: 100%|██████████| 15102/15102 [36:22<00:00,  6.92it/s, Batch Loss=0.11]   


Epoch 4 completed. Average Loss: 0.06390370413724646


Epoch 5: 100%|██████████| 15102/15102 [36:21<00:00,  6.92it/s, Batch Loss=0.0693] 


Epoch 5 completed. Average Loss: 0.06413668885206923


('./kogpt2-korquad-finetuned/tokenizer_config.json',
 './kogpt2-korquad-finetuned/special_tokens_map.json',
 './kogpt2-korquad-finetuned/vocab.json',
 './kogpt2-korquad-finetuned/merges.txt',
 './kogpt2-korquad-finetuned/added_tokens.json',
 './kogpt2-korquad-finetuned/tokenizer.json')

In [33]:

question = "인공지능이란?"
context = "인공지능은 1950년대 부터 연구가 시작되었으며, 심리학과 컴퓨터 과학의 융합으로 발전해왔다."

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=400,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")

질문: 인공지능이란?
컨텍스트: 인공지능은 1950년대 부터 연구가 시작되었으며, 심리학과 컴퓨터 과학의 융합으로 발전해왔다.
생성된 답변: <usr> 인공지능이란? <sys> 인공지능은 1950년대 부터 연구가 시작되었으며, 심리학과 컴퓨터 과학의 융합으로 발전해왔다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [39]:
model.eval()
for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                max_length=513,
                repetition_penalty=2.0,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
            )
        
        print(f"질문: {tokenizer.decode(input_ids[0], skip_special_tokens=False)}")
        print(f"생성된 답변: {tokenizer.decode(output_ids[0], skip_special_tokens=False)}")
        break

Epoch 1:   0%|          | 0/15102 [00:00<?, ?it/s]


질문: <usr> 1920년대 경제학에서 미시경제학과 거시경제학을 분리하여 볼 것을 제안한 인물은? <sys> 1920년대에 케인스는 경제학에서 미시경제학과 거시경제학을 분리하여 볼 것을 제안하였다. 케인스 경제학의 거시경제학은 개인의 선택에 절대적으로 의존하고 있던 당시 고전경제학의 문제를 극복할 수 있었다. 이 이론에 따라, 정부는 상품시장의 총수요(Aggregate demand)를 자극하여 경제 성장을 촉진해게 되었다. 세계제2차대전 이후에 프리드먼은 통화주의(Monetarism)의 개념을 제시하였다. 통화주의는 통화가 경제 활동을 통제하게된다는 관점에서 통화의 수요와 공급을 주목하게 하였다. 1970년대 들어, 통화주의는 서플라이사이드이코노믹스(Supply-side economics)로 발전하였다. 서플라이사이드이코노믹스는 감세가 통화량을 증가시켜 경제성장을 촉진하게 된다는 이론이다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

Epoch 2:   0%|          | 0/15102 [00:00<?, ?it/s]


질문: <usr> 미혼부가 아이의 아빠임이 증명되면 출생신고를 할 수 있게 하는 법을 대표발의한 인물은? <sys> 서영교는 사랑이 친부와의 면담 등을 통해 미혼부의 어려움을 듣고 미혼부도 DNA검사 등을 통해 아이가 아빠의 아이라는 사실이 증명되면 법원에 신청을 통해 아이의 출생신고가 가능하게 하는 법을 대표발의했고, 2015년 4월 30일 국회 본회의에서 수정 가결 되었으며 2015년 11월 19일부터 시행중이다. 현재 법원은 친생자 출생신고를 위한 확인 신청서 에서 '모의 성명·등록기준지·주민등록번호를 알 수 없는 사유를 소명할 수 있는 자료'의 첨부를 요구하고 있다. 사랑이 아버지는 그가 낸 1심 소송에서 법원이 이를 세가지 사항을 모두 몰라야 출생신고가 가능하다고 결정하여 출생신고를 하지 못했으나, 그러나 법원은 친모 인적사항인 '모의 성명·등록기준지·주민등록번호'의 3요소중 하나를 모르면 법원 확인 방식의 미혼부 출생신고가 가능하다고 판결했다. 원래 혼인외 출생자의 경우에도 부가 모의 성명, 등록기준지, 주민등록번호를 모두 알고 있으면 출생신고가 수리됐다. 2심법원은 "이 법에서 '모의 성명·등록기준지·주민등록번호를 알 수 없는 경우'라 함은 인적사항을 전부 또는 일부를 알 수 없어 친모를 특정하지 못해 친부 혼자 출생신고를 못 하는 경우를 의미한다고 해석해야 한다"고 결정했다. 이에 따라 법원은 사랑이가 법에서 '모의 성명·등록기준지·주민등록번호를 알 수 없는 경우'에 놓여있다고 결정했다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

Epoch 3:   0%|          | 0/15102 [00:00<?, ?it/s]


질문: <usr> 759년 경에 일본 최고의 시집으로 등장한 것은 무엇인가? <sys> 일본 문학의 기원은 구전 문학(口傳文學)에서 비롯하여 8세기 초에 최초의 기록 문학 형태를 갖추었다. 왕실에서 직접 주도하여 712년 신화와 전설에 관한 작품집인 고지키가, 720년에는 역사서인 니혼쇼키가 완성되었다. 또 759년 경에는 일본 최고(最古)의 시집으로 4,500편의 시가 수록된 만요슈가 등장하여 이 시기에 단가(短歌) 형태의 시가 문학이 나타났음을 입증하고 있다. 그 후 헤이안 시대의 귀족 문화에서 문학의 중심적인 역할을 담당했던 사람들은 궁중의 여관(女官)으로, 이 시기에 세이 쇼나곤, 무라사키 시키부 등의 여성 시인이 활발하게 활동한다. 센고쿠 시대와 에도 시대를 거쳐서 17세기에는 마쓰오 바쇼가 17자의 단문 시 형식인 하이쿠를 만들기도 하였다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Epoch 4:   0%|          | 0/15102 [00:00<?, ?it/s]


질문: <usr> 3세기와 4세기 사이에 발전한 정권의 이름은? <sys> 기원전 8세기 경 이후 대륙으로부터 벼농사를 중심으로 한 문화 양식이 전해지면서 각지에 ‘무라’, ‘구니’와 같은 정치 조직이 천천히 형성되어 1세기 ~ 2세기 전후에는 각 구니의 연합체로서 왜국이라는 대규모 정치 조직이 출현했다. 이 연합 정치 조직은 3세기에서 4세기 사이에 야마토 정권으로 발전했다. 그러다 663년 백제 부흥 운동에서 백제를 도왔다가 신라와 당나라에 패배하고 백제의 지배 계층이 대량 유입되어 7세기 후반 한자 문화권에 본격적으로 흡수되어 중국식 법체계, 사회 제도를 급속도로 받아들여 8세기 초 고대 율령 국가 체제가 완성되기에 이르렀다. 이 시기 일본은 수나라, 당나라나 신라, 발해 들과 교류 이후 주변국으로부터 많은 문화를 받아들였고, 이를 바탕으로 10세기에서 11세기 사이에 장원을 중심으로 한 봉건 체제를 바탕으로 귀족 문화를 형성하였으며 귀족들의 세력 투쟁 사이에서 발달한 사무라이 계층은 13세기 이후 가마쿠라 막부를 세워 정치적 주도권을 잡았다. 한편으로는 무로마치 시대에서 센고쿠 시대까지 명나라에 조공을 행하였으며 쇼군 가문인 아시카가 씨가 명나라 천자로부터 일본 국왕으로서 책봉을 받기도 하였다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Epoch 5:   0%|          | 0/15102 [00:00<?, ?it/s]

질문: <usr> 파워 포인트를 소모해서 사용하는 사이오닉 파워는 어떤 클래스만 사용하는가? <sys> 2003년 초 플레이어 리소스 컨소시엄(Player Resource Consortium, PRC)이 게임에 클래스, 종족, 기술, 주문을 추가해 주는 핵 팩 묶음인 PRC를 선보였다. 2006년 5월 20일 현재, PRC는 오리지널 게임에 있었던 프레스티지 클래스 보다 최소한 세 배가 많은 클래스를 포함하고 있다. 또한 PRC는 바이오웨어의 오로라 엔진을 더 잘 사용하는 여러 에픽 주문들과 많은 보통 주문들을 추가해 준다. 추가된 주문들은 순간이동, 전위, 미로, 말 소환 등이다. 또한 본질적으로는 주문이지만, 파워 포인트를 소모하며 소서러 클래스만 사용할 수 있는 사이오닉 파워도 추가됐다. PRC는 대개 엔진 디자이너가 의도하지 못했던 방법으로 게임 엔진을 몰아붙이기 때문에, 이 핵 팩을 사용할 때에는 주의가 필요하다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad